In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path.append("../src")
import os
from src.DS_Generator import DataSet_Generator
from src.model_evaluation import mean_average_precision
from data_augmentation import *
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from util import *
import datetime
%load_ext tensorboard

Okay Louis, such dir eine Batchsize aus. Bei mir hat nur 32 keine fehler verusacht.

In [3]:
# Für deinen Model order
if "models" not in os.listdir("../../"):
    os.makedirs("../../models")
    
BATCH_SIZE = 32

In [4]:
train_ds, val_ds = DataSet_Generator().generate_species_data(TRAIN_SPECIES_DF,augment=1,batch_size=BATCH_SIZE)
num_classes = len(set(TRAIN_SPECIES_DF["species"]))

We threw away the datapoint with index 50035 


In [7]:
Input = tf.keras.Input((224,224,3))
base = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet", include_top=False,input_tensor=Input)
flatten = base.output
flatten = tf.keras.layers.Flatten()(flatten)
head = tf.keras.layers.Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(flatten)
head = tf.keras.layers.Dense(128, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(head)
head = tf.keras.layers.Dense(64, activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.001))(head)
head = tf.keras.layers.Dense(num_classes,activation="softmax")(head)
model = tf.keras.Model(inputs=Input, outputs=head,name="Inception_v3_with_imagenet_weights_and_l2")

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss="categorical_crossentropy",metrics=["acc"])

if model.name not in os.listdir("../../models/"):
    os.makedirs("../../models/"+model.name)
    os.makedirs("../../models/"+model.name+"/logs")
    os.makedirs("../../models/"+model.name+"/saves")
    
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = "../../models/"+model.name+"/logs/" +time_stamp 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "../../models/"+model.name+"/saves/" +time_stamp +"/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [8]:
model.fit(
train_ds,
epochs=15,
validation_data=val_ds,
callbacks=[cp_callback,tensorboard_callback])

Epoch 1/15


UnimplementedError: Graph execution error:

Detected at node 'Inception_v3_with_imagenet_weights_and_l2/conv2d_94/Conv2D' defined at (most recent call last):
    File "C:\Users\Felix\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Felix\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Felix\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\Felix\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\Felix\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\Felix\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File